# Message Memory in Agent backed by a database
---

Este cuaderno explica cómo agregar memoria a un Agente donde la memoria utiliza un almacén de mensajes externo. Antes de revisar este cuaderno, lea los siguientes cuadernos, ya que se basarán en ambos:

- [Memory in LLMChain](./memory_in_llm_chain.ipynb)
- [Custom Agent](../agents/how_to/custom_agent.ipynb)
- [Memory in Agent](./memory_in_agent.ipynb)

Para añadir una memoria con un almacén de mensajes externo a un agente vamos a realizar los siguientes pasos:

1. Vamos a crear un `RedisChatMessageHistory` para conectarnos a una base de datos externa para almacenar los mensajes.

2. Vamos a crear una `LLMChain` usando ese historial de chat como memoria.

3. Usaremos ese `LLMChain` para crear un Agente personalizado.


Para los fines de este ejercicio, crearemos un Agente personalizado simple que tenga acceso a una herramienta de búsqueda y utilice la clase `ConversationBufferMemory`.

In [3]:
from dotenv import load_dotenv, find_dotenv
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.utilities import SerpAPIWrapper

load_dotenv(find_dotenv())

True

In [4]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

Observe el uso de la variable `chat_history` en `PromptTemplate`, que coincide con el nombre de la clave dinámica en `ConversationBufferMemory`.

In [5]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

Ahora podemos crear `RedisChatMessageHistory` respaldado por la base de datos.

In [7]:
message_history = RedisChatMessageHistory(
    url="redis://localhost:6379/0",
    ttl=600,
    session_id="my-session"
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=message_history
)

Ahora podemos construir `LLMChain`, con el objeto Memoria, y luego crear el agente

In [8]:
# llm
llm = OpenAI(temperature=0)

# llm chain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
)

# agent
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    verbose=True,
)

# agent chain
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
)

In [10]:
agent_chain.run(input="How many people live in canada in 2023")



> Entering new AgentExecutor chain...
Thought: I need to find out the population of Canada in 2023
Action: Search
Action Input: Population of Canada in 2023
Observation: 38,781,291
Thought: I now know the final answer
Final Answer: 38,781,291 people will live in Canada in 2023.

> Finished chain.


'38,781,291 people will live in Canada in 2023.'

Para probar la memoria de este agente, podemos hacer una pregunta de seguimiento que se base en la información del intercambio anterior para ser respondida correctamente.

In [11]:
agent_chain.run(input="what is their national anthem called?")



> Entering new AgentExecutor chain...
Thought: I need to find out what the national anthem of Canada is called.
Action: Search
Action Input: National anthem of Canada
Observation: ['"O Canada" is the national anthem of Canada. The song was originally commissioned by Lieutenant Governor of Quebec Théodore Robitaille for the 1880 Saint-Jean-Baptiste Day ceremony; Calixa Lavallée composed the music, after which words were written by the poet and judge Sir Adolphe-Basile Routhier.', 'O Canada type: National anthem.', 'O Canada main_tab_text: Overview.', 'O Canada kgmid: /m/05mtn.', 'O Canada lyricists: Calixa Lavallée, Robert Stanley Weir, Adolphe-Basile Routhier.', 'O Canada composer: Calixa Lavallée.', 'O Canada key: G major.', 'O Canada adopted: July 1, 1980.', 'O Canada also_known_as: French - Ô Canada.', 'It was proclaimed the official national anthem on July 1, 1980. “God Save the Queen” remains the royal anthem of Canada. The music, written by ...']
Thought: I now know the final a

'The national anthem of Canada is called "O Canada".'